In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import joblib
from joblib import Parallel, delayed
import multiprocessing

# Load the dataset from your local machine
file_path = r'C:\Users\Riphah\Desktop\IoT_Intrusion.csv'
print("Loading dataset...")
data = pd.read_csv(file_path)
print("Dataset loaded successfully.")

# Separate features and target variable
X = data.drop(['label'], axis=1)
y = data['label']

# Split the data into training and testing sets
print("Splitting data into training and testing sets...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Data split completed.")

# Get the number of available cores
num_cores = multiprocessing.cpu_count()

# Train SVM model using full CPU utilization with parallel processing
print("Training SVM model with full CPU utilization and parallel processing...")
svm_model = SVC(kernel='rbf', random_state=42, probability=True, verbose=True, max_iter=1000)

# Define a function to train SVM model on a chunk of data
def train_svm_model(chunk_X, chunk_y):
    svm_model.fit(chunk_X, chunk_y)

# Divide the data into chunks based on the number of available cores
chunks_X = [X_train[i::num_cores] for i in range(num_cores)]
chunks_y = [y_train[i::num_cores] for i in range(num_cores)]

# Train SVM model on each chunk of data in parallel
Parallel(n_jobs=num_cores)(delayed(train_svm_model)(chunk_X, chunk_y) for chunk_X, chunk_y in zip(chunks_X, chunks_y))

print("Model trained successfully.")

# Cross-validation
print("Performing cross-validation...")
cv_scores = cross_val_score(svm_model, X_train, y_train, cv=5, scoring='accuracy', n_jobs=-1)
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean CV Accuracy: {cv_scores.mean():.4f}")

# Evaluate the model
print("Evaluating the model...")
y_pred = svm_model.predict(X_test)

# Calculate evaluation metrics
conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Evaluation completed.")
print("Confusion Matrix:")
print(conf_matrix)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

# Save the trained model
print("Saving the trained model...")
joblib.dump(svm_model, 'svm_model.h5')
print("Model saved successfully.")
